Imports


In [17]:
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, LSTM, Bidirectional
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
import gensim
from gensim.models import Word2Vec
import tensorflow.keras as keras
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
nltk.download('stopwords')
import re
from sklearn.model_selection import train_test_split
from keras.layers import SimpleRNN
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Data Cleaning (preprocessing)


In [18]:


text_cleaning_re = "@\S+|https?:\S+|http?:\S+|[^A-Za-z0-9]:\S+|subject:\S+|nbsp"
def preprocess(text, stem=False):
    stemmer = PorterStemmer()
    text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

train_dataset = pd.read_csv("spam_ham_dataset.csv")
stop_words = stopwords.words('english')

y_train = train_dataset['label_num']
x_train = train_dataset['text']


#Data Preprocessing: (word stem)

stemmed_words_x = x_train.apply(lambda x: preprocess(x,True))
#Wor2vec part/ tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(stemmed_words_x)

word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 10000
# print("Vocabulary Size :", vocab_size)

stemmed_words_x.head().reset_index()
full_x_train = pad_sequences(tokenizer.texts_to_sequences(stemmed_words_x),maxlen = 50)
full_y_train = y_train
x_train, x_train2, y_train, y_train2 = train_test_split(full_x_train, y_train, test_size=0.3, random_state=0)
x_train, x_train3, y_train, y_train3 = train_test_split(x_train, y_train, test_size=0.3, random_state=0)


Model Training -LSTM-

In [19]:

MAX_SEQUENCE_LENGTH = 50
n_lstm = 200
drop_lstm = 0.2
embeding_dim = 16
drop_value = 0.2
n_dense = 24
num_epochs = 10
early_stop = EarlyStopping(monitor='val_loss', patience=3)
#LSTM Spam detection architecture
model1 = Sequential()
model1.add(Embedding(vocab_size, embeding_dim, input_length=MAX_SEQUENCE_LENGTH))
model1.add(LSTM(n_lstm, dropout=drop_lstm, return_sequences=True))
model1.add(LSTM(n_lstm, dropout=drop_lstm, return_sequences=True))
model1.add(Dropout(0.1))
model1.add(Dense(1, activation='relu'))
model1.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
# model1.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'],validation_data=(x_valid,y_valid))
print(x_train.shape)

history = model1.fit(x_train, y_train, epochs=num_epochs, verbose=2)
early_stop = EarlyStopping(monitor='val_loss', patience=2)


(2533, 50)
Epoch 1/10
80/80 - 24s - loss: 0.5808 - accuracy: 0.7358 - 24s/epoch - 301ms/step
Epoch 2/10
80/80 - 21s - loss: 0.2936 - accuracy: 0.8628 - 21s/epoch - 261ms/step
Epoch 3/10
80/80 - 20s - loss: 0.2144 - accuracy: 0.9018 - 20s/epoch - 255ms/step
Epoch 4/10
80/80 - 20s - loss: 0.1940 - accuracy: 0.9096 - 20s/epoch - 256ms/step
Epoch 5/10
80/80 - 21s - loss: 0.1746 - accuracy: 0.9163 - 21s/epoch - 257ms/step
Epoch 6/10
80/80 - 21s - loss: 0.1682 - accuracy: 0.9192 - 21s/epoch - 257ms/step
Epoch 7/10
80/80 - 21s - loss: 0.1609 - accuracy: 0.9224 - 21s/epoch - 266ms/step
Epoch 8/10
80/80 - 21s - loss: 0.1536 - accuracy: 0.9273 - 21s/epoch - 260ms/step
Epoch 9/10
80/80 - 21s - loss: 0.1764 - accuracy: 0.9173 - 21s/epoch - 260ms/step
Epoch 10/10
80/80 - 21s - loss: 0.1511 - accuracy: 0.9281 - 21s/epoch - 258ms/step


Training of the model 5 -Bi LSTM-


In [20]:
BiLSTM = Sequential()
BiLSTM.add(Embedding(vocab_size, embeding_dim, input_length=MAX_SEQUENCE_LENGTH))
BiLSTM.add(Bidirectional(LSTM(n_lstm, dropout=drop_lstm, return_sequences=True)))
BiLSTM.add(Dense(1, activation='sigmoid'))
BiLSTM.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
# Training
num_epochs = 5
early_stop = EarlyStopping(monitor='val_loss', patience=2)
history = BiLSTM.fit(full_x_train, full_y_train, epochs=num_epochs, 
                    callbacks =[early_stop], verbose=2)

Epoch 1/5
162/162 - 34s - loss: 0.2562 - accuracy: 0.8930 - 34s/epoch - 212ms/step
Epoch 2/5
162/162 - 31s - loss: 0.0723 - accuracy: 0.9831 - 31s/epoch - 191ms/step
Epoch 3/5
162/162 - 31s - loss: 0.0177 - accuracy: 0.9964 - 31s/epoch - 189ms/step
Epoch 4/5
162/162 - 31s - loss: 0.0099 - accuracy: 0.9981 - 31s/epoch - 189ms/step
Epoch 5/5
162/162 - 31s - loss: 0.0117 - accuracy: 0.9975 - 31s/epoch - 189ms/step


Training of Model 2 -RNN-

In [21]:
rnn = Sequential()
rnn.add(SimpleRNN(128, activation='relu', input_dim=50 , return_sequences = True))
rnn.add(SimpleRNN(64, activation='relu' ,  return_sequences = False))
# Adding dropout to prevent overfitting
#rnn.add(Dropout(0.1))
rnn.add(Dense(1, activation='sigmoid'))
rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

x_train2 = x_train2.reshape(len(y_train2),1,50)
y_train2 = np.array(y_train2).reshape(len(y_train2),1,1)
rnn.fit(x_train2, y_train2, batch_size=100, epochs=150)

Epoch 1/150
16/16 [==============================] - 2s 4ms/step - loss: 529.9043 - accuracy: 0.5664
Epoch 2/150
16/16 [==============================] - 0s 4ms/step - loss: 249.6195 - accuracy: 0.6044
Epoch 3/150
16/16 [==============================] - 0s 5ms/step - loss: 156.0589 - accuracy: 0.6649
Epoch 4/150
16/16 [==============================] - 0s 4ms/step - loss: 97.7191 - accuracy: 0.6765
Epoch 5/150
16/16 [==============================] - 0s 4ms/step - loss: 67.1199 - accuracy: 0.7313
Epoch 6/150
16/16 [==============================] - 0s 4ms/step - loss: 52.2584 - accuracy: 0.7481
Epoch 7/150
16/16 [==============================] - 0s 4ms/step - loss: 36.9000 - accuracy: 0.7816
Epoch 8/150
16/16 [==============================] - 0s 5ms/step - loss: 24.6185 - accuracy: 0.8073
Epoch 9/150
16/16 [==============================] - 0s 4ms/step - loss: 21.3814 - accuracy: 0.8247
Epoch 10/150
16/16 [==============================] - 0s 3ms/step - loss: 18.1470 - accuracy: 0.8

Training of Model 3 -MNNB-

In [22]:
# Create the model with the best hyperparameters

parameters = {"alpha": [0.2,1,2,5,10], "fit_prior": [True, False]}

grid = GridSearchCV(MultinomialNB(), param_grid=parameters)
grid.fit(x_train3,y_train3)

# Create a DataFrame with the best Hyperparameters
pd.DataFrame(grid.cv_results_)[['params','mean_test_score']]\
                               .sort_values(by="mean_test_score", ascending=False)
grid.best_params_
alpha, fit_prior = grid.best_params_['alpha'], grid.best_params_['fit_prior']
model = MultinomialNB(alpha = alpha)

model.fit(x_train3,y_train3)
# y_pred = model.predict(X_test)

#

MultinomialNB(alpha=0.2)

Training of Model 4 -XGBoost-

In [23]:
xgb = XGBClassifier()
xgb.fit(full_x_train, full_y_train)


XGBClassifier()

In [27]:



test_dataset = pd.read_csv("messages.csv",encoding='latin-1')

y_test = [True if y == 'spam' else False for y in test_dataset["label"]]
x_test = test_dataset["message"]

#Data Preprocessing: (word stem)

x_test = x_test.apply(lambda x: preprocess(x,True))
#Wor2vec part/ tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_test)

word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1000
# print("Vocabulary Size :", vocab_size)

x_test.head().reset_index()
x_test = pad_sequences(tokenizer.texts_to_sequences(x_test),maxlen = 50)


# model1.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'],validation_data=(x_test,y_test))
y1 = model1.predict(x_test)
var1=(y1>0.5)
x_test1 = x_test.reshape(len(y_test),1,50)

y2 = rnn.predict(x_test1)
var2=(y2>0.5)

y3 = model.predict(x_test)
var3=(y3>0.5)

y4 = xgb.predict(x_test)
var4 = (y4>0.5)

y5 = BiLSTM.predict(x_test)
var5 = (y5>0.5)

phase de Test

In [35]:
cpt = 0
tot = len(var1)
print(y1.shape)
print(len(y_test))

for i in range(len(var1)):
  if list(var1[i]).count([y_test[i]])/50 > 0.10 :
    cpt += list(var1[i]).count([y_test[i]])/50 > 0.50 
  
  # print(var1[i][0])
print("LSTM: ", cpt/tot)

cpt = 0
tot = len(var5)
print(y5.shape)
print(len(y_test))

for i in range(len(var5)):
  if list(var5[i]).count([y_test[i]])/50 > 0.10 :
    cpt += list(var5[i]).count([y_test[i]])/50 > 0.50 
  
  # print(var1[i][0])
print("BiLSTM: ", cpt/tot)

cpt = 0
tot = len(var2)
print(y2.shape)

for i in range(len(var2)):
  if var2[i][0] == y_test[i] :
    cpt += 1
  
  # print(var1[i][0])
print("RNN: ", cpt/tot)


cpt = 0
tot = len(var3)
print(y3.shape)
for i in range(len(var3)):
  if var3[i] == y_test[i] :
    cpt += 1
  # print(var1[i][0])
print("MNB: ", cpt/tot)

cpt = 0
tot = len(var4)
print(y4.shape)
for i in range(len(var4)):
  if var4[i] == y_test[i] :
    cpt += 1
  # print(var1[i][0])
print("XGBoost: ", cpt/tot)

EL_cpt = 0
FP = 0
FN = 0
for i in range(len(y_test)):
  el = [y_test[i] if list(var1[i]).count([y_test[i]])/50 > 0.50 else not y_test[i],var2[i][0],var3[i],var4[i],y_test[i] if list(var5[i]).count([y_test[i]])/50 > 0.50 else not y_test[i]]

  if(el.count(y_test[i]) >= 3): EL_cpt += 1
  elif(el.count(False) >= 3 and y_test[i]): FN +=1
  elif(el.count(True) >= 3 and  not y_test[i]): FP +=1
print("EL: ",EL_cpt/tot)
print("EL False Negative: ",FN/tot)
print("EL: False Positive",FP/tot)


(2893, 50, 1)
2893
LSTM:  0.9170411337711718
(2893, 50, 1)
2893
BiLSTM:  0.9246456965088143
(2893, 1)
RNN:  0.6539923954372624
(2893,)
MNB:  0.4870376771517456
(2893,)
XGBoost:  0.594192879363982
EL:  0.8420324922226063
EL False Negative:  0.0
EL: False Positive 0.1579675077773937
